In [ ]:
import logging
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.local import LocalSession

#No need to be signed in (eg.g. SSO) when using LocalSession local code mode
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}
DUMMY_IAM_ROLE = 'arn:aws:iam::999999999999:role/dummy-role/AmazonSageMaker-ExecutionRole-20000101T000000'

train = "file://local/data"
validation = "file://local/data"
output = "file:///tmp/sagemaker-xgboost-local"

hyperparams = {
    "eta": 0.1,
    "num_class": 30,
    "num_round": 20,
    "objective": "multi:softprob",
    "hite_rate_k": 5
}

estimator = XGBoost(
    entry_point="train.py",
    hyperparameters=hyperparams,
    role=DUMMY_IAM_ROLE,
    instance_count=1,
    instance_type="local",
    framework_version="1.3-1",
    output_path=output,
    sagemaker_session=sagemaker_session
)

estimator.fit(
    inputs = {
        "train": train,
        "validation": validation
    }
)

# since the train.py also includes the code to load the model during inference
# we can deploy the estimator locally immediatelly, it knows where the model.tar.gz exists.
# Training and inference 
predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='local'
)

payload = '1, 0.455, 0.365, 0.095, 0.514, 0.2245, 0.101, 0.15'
preds = predictor.predict(payload)

logging.info(f"prediction: {preds}")
